# mBuild Tutorial 03: Connecting Components with Ports

This tutorial demonstrates how to create `Ports` on `Compounds`, which aids in connecting them together by both creating bonds and moving the `Compounds` in space.

As in the prior tutorials, we need to first import mbuild (here as `mb`).

In [ ]:
import mbuild as mb

We've already shown that `Particles` can be connected (i.e. bonded) by using the `add_bond` routine; however, this does not actually move the atoms in space, and it would become burdensome to need to manually update the position of each atom when connecting them together. Additionally, having to create fixed bonds for all connections would make mBuild less flexible; again, a key function of mBuild is the ability to exchange and swap `Compounds`, allowing arbitrary molecules to be constructed.  

This is where [mBuild's `Port` class](http://mosdef-hub.github.io/mbuild/data_structures.html#mbuild.port.Port) comes into play. `Ports`, in the most general sense, define a location (and direction) in space; however, in most cases these can be thought of as dangling bonds.

Let's test this functionality by using `Ports` instead of `add_bond` to create a ch2 moiety. First, we'll create an empty `Compound` for CH2 that we will add three `Particles` to at unrealistic locations.

In [ ]:
ch2 = mb.Compound()
carbon = mb.Particle(pos=[0.0, 0.0, 0.0], name='C')
hydrogen = mb.Particle(pos=[1.0, 0.0, 0.0], name='H')
hydrogen2 = mb.Particle(pos=[2.0, 0.0, 0.0], name='H')
ch2.add([carbon, hydrogen, hydrogen2])
ch2.visualize()

Now we'll create a `Port` instance and attach it to the carbon atom by using the `anchor` attribute. This allows mBuild to know which atoms to create bonds between when two `Ports` are connected (as well as providing a reference for any geometric transformations). We can also provide an `orientation` vector to give our `Port` a desired direction, and can use the `separation` argument to shift our `Port` from the position of the anchor `Particle`. Since we're going to be connecting to a hydrogen, we will shift our `Port` roughly half of a C-H bond length.

In [ ]:
port_C = mb.Port(anchor=carbon, orientation=[1, 0, 0], separation=0.05)
port_C

We now need to add this `Port` to the containment hierarchy of our ch2 `Compound`, again using the `add` method. We can also provide a descriptive label for our `Port` that we can use for easy access; here we will name this port `right`.

In [ ]:
ch2.add(port_C, label='right')
ch2['right']

Now we need to add another `Port` to the carbon `Particle` and one `Port` to each hydrogen `Particle`, giving each of these distinct labels. We'll first add another `Port` to carbon (labeled `left`) and a `Port` on each of the hydrogens (labeled `H1` and `H2`).  

In [ ]:
port2_C = mb.Port(anchor=carbon, orientation=[-1, 0, 0], separation=0.05)
ch2.add(port2_C, label='left')

port_H = mb.Port(anchor=hydrogen, orientation=[1, 0, 0], separation=0.05)
ch2.add(port_H, label='H1')

port2_H = mb.Port(anchor=hydrogen2, orientation=[1, 0, 0], separation=0.05)
ch2.add(port2_H, label='H2')

ch2.visualize(show_ports=True)

The `force_overlap` function is then used to force the overlap of two `Ports` by performing a coordinate transform on one of the two `Compounds` that are to be connected. This will also create a bond between the anchor `Particles` of each `Port`. We'll use this function here to connect each hydrogen to the carbon `Particle`.

In [ ]:
mb.force_overlap(move_this=hydrogen,
                 from_positions=ch2['H1'],
                 to_positions=ch2['right'])

mb.force_overlap(move_this=hydrogen2,
                 from_positions=ch2['H2'],
                 to_positions=ch2['left'])

ch2.visualize(show_ports=True)

As can be seen above, the hydrogen atoms were appropriately translated and oriented to create the expected ch2 structure.

Note that once two `Particles` are connected using `force_overlap` the `Ports` used to connect these `Particles` are removed. However, if we were to remove the bond between one of the hydrogen atoms and the carbon atoms, two new `Ports` will be created, one attached to each `Particle`, along the bond vector.

In [ ]:
ch2.remove_bond(particle_pair=(carbon, hydrogen))
ch2.visualize(show_ports=True)

We could now remove the hydrogen we disconnected and connect something else to our CH moiety. To achieve this, we will need to use the `remove` method to remove both the hydrogen atom *and* the `Port` connected to this atom from our molecule hierarchy.

First, however, we'll use the `available_ports` method to query the `Ports` present in our molecule, so that we can fine out the name of the `Port` we want to remove.

In [ ]:
ch2.available_ports()

Now we'll use `remove` to remove `port[1]` and the hydrogen atom from our molecule.

In [ ]:
ch2.remove([ch2['port[1]'], hydrogen])
ch2.visualize(show_ports=True)

Now, we'll use the `clone` function to create a deep copy of the CH moiety and, again using `force_overlap` will connect the two CH's together. We'll then add both of these CH moieties to a parent `Compound` and visualize.

In [ ]:
ch2_copy = mb.clone(ch2)
mb.force_overlap(move_this=ch2_copy,
                 from_positions=ch2_copy['port[0]'],
                 to_positions=ch2['port[0]'])
parent = mb.Compound(subcompounds=[ch2, ch2_copy])
parent.visualize(show_ports=True)

We now see that we have two CH's connected! However, because our `Ports` were based off of the `C-H` bond distance, the two carbon atoms are unrealistically close to one another. While mBuild does feature an `energy_minimization` function that could remedy this, we could also manually shift our CH2 copy a bit to the left using the `translate` method.

In [ ]:
ch2_copy.translate([0.05, 0, 0])
parent.visualize(show_ports=True)

## Recap

The goal of this tutorial was to demonstrate the use of mBuild's `Port` class to aid in connecting particles and molecules as well as moving them around in space. We also explored several additional methods of `Compound` including `translate`, `available_ports`, and `remove_bond`.

The next tutorial will take the routines we've learned and use them to create a more complex molecule, an alkane chain.